# EmergencyZIP AI

## Intro

As a Data Scientist in Healthcare with 2+ years of industry and work experience, one of the valuable pieces of knowledge that I learned on the job is that Zipcode (Postal Code) is indeed an important factor in determining an individual's health status (thus, wherever the individual lives can determine how healthy or unhealthy an individual can be). Also, healthcare, especially the quality of healthcare, is also an important factor in determining the standard of life of an area as well. However, many people may not be familiar with all the medical facilities that their area may provide or may not know the best one to go to based on what illness or medical condition that they may experience now. Thus, I created the EmergencyZIP AI to help individuals determine the best medical facilities to attend based on the symptoms or conditions that they may be experiencing right now, and to take greater consideration of zipcode/postal code as one of the most important factors of an individual's health status. 

## Use Case and Solution Approach

Individuals may want to know or curious about what current illness or medical condition that they may be facing at the moment, and the nearest and most appropriate medical facilities in their area that can best treat their illness or condition. I created EmergencyZip AI to utilize AI capabilities to not only inform the individual of what illness or condition that they may be facing now, but also inform the individual of the nearest appropriate medical facilities that can best treat the patient with great consideration of the zipcode/postal code as one of the most important factors of an individual's health status. 

## Process

The AI utilizes Structured Output/JSON Mode/Controlled Generation, Few-Shot Prompting, and Grounding to operate. First, it uses JSON Mode to process patient demographic information and structures it to a formalized JSON form (just like filling out a patient form to receive medical care except simplier and more abstract). Then it utilizes Few-Shot Prompting and Structured Output/Controlled Generation to create a formalized request for the AI. Then it utilizes grounding to provide the best and most accurate potential diagnosis for the individual based on the current symptoms/conditions that the individual is facing now. Then using the diagnosis and the current zipcode of the individaul (the AI uses Few-Shot Prompting to extract the zipcode of the individual from the JSON form), the AI finds the nearest and best medical facilities that can treat the individual for the diagnosed illness or condition using grounding. The AI also utilizes grounding to inform the individual payment and insurance plan options that the suggested medical facilities can accept as payment for the treatment. 

## Innovation/Novelty

While the internet is a powerful tool to use to research medical facilities available in the area, with so many search results provided, it can be messy and overwhelming at times to find the best medical facility for treatment of illnesses and medical conditions or even know what actual/potential illness or medical condition that the individual may be facing. With EmergencyZIP AI, the responses provided are tailored and focused only on the individual and the potential illness or medical condition that they are facing now. Also while doctors usually do a great job with diagnosing patients, there could be human bias or error that can be involved in the diagnosis process, so the AI takes in consideration of these potential biases (taking in consideration of patient demographic information) and gives the most accurate diagnosis as possible without being biased. 

### Importing Relevant Libraries and Technologies 

I set up Gemini API to implement my AI (the code also retrieves the API Key from Kaggle Secrets to retrieve the Google API that makes it possible for the AI to work) and imported the Pandas and Random libraries (and other unnamed libraries) to do my test run to check if the AI works as intended or not. 

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import random

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/patient-symptom-dataset/patient_symptom_data.csv


The cell below removes any irrelevant libraries that may conflict with the Google Generative AI capabilities and install the Google Gemini Generative AI package

In [2]:
!pip uninstall -qqy jupyterlab  # Remove unused packages from Kaggle's base image that conflict
!pip install -U -q "google-genai==1.7.0"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.7/144.7 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.9/100.9 kB 4.7 MB/s eta 0:00:00


Install all relevant libraries for the Google Generative AI capabilities

In [3]:
from google import genai
from google.genai import types

from IPython.display import HTML, Markdown, display

In [4]:
from google.api_core import retry


is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})

genai.models.Models.generate_content = retry.Retry(
    predicate=is_retriable)(genai.models.Models.generate_content)

Retrieve the Google API Key from Kaggle Secrets to make possible for the AI to work in this code.

In [5]:
from kaggle_secrets import UserSecretsClient

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")

In [6]:
client = genai.Client(api_key=GOOGLE_API_KEY)

## Generative AI Capabilities Used and Reasoning 

These are the 3 Generative AI Capabilities Used in this AI and why they are used in this AI: 
* **Structured Output/JSON Mode/Controlled Generation**: For JSON Mode, I want to somewhat mimic the forms that patients fill out when they are requesting care from a medical facility, but simplier and more abstract. For Structured Output and Controlled Generation, I want my responses to be as accurate and precise as possible where creativity may not be required but accuracy is definitely required to get the best responses as possible.
* **Few-Shot Prompting**: First of all, I want to create requests for the AI to know who it is providing information for, but I want the formatting of the requests to be as consistent as possible so that the information does not get muddled up which would make things more difficult for the AI to process and provide information for. Then, I also want to extract the zipcode from the JSON form, but sometimes the AI can hallucinate and return irrelevant or incorrect information, so I want the let the AI know how to return the zipcode correctly.
* **Grounding**: This is the central capability of the AI since the AI is responsible for finding all the relevant possible nearest locations to the patient based on postal code/zipcode. Since we are exploring so many zipcodes and it is possible that the AI does not know every zipcode or medical facilities per zipcode, we need to rely on external sources to get the best answers possible. However, since we don't have a document listing out every medical facility per zipcode or all zipcodes, we would need to rely on the internet to give us answers. 

### Generative AI Capability #1: Structured Output/JSON Mode/Controlled Generation

#### JSON Mode

I created a Python class MedicalPatients to format the JSON form as a simple and abstract patient form, and then I created a get_json_form function that takes in an entry (in this test case, it's a Pandas dataframe entry/row) and structure it to a JSON form as structured in the MedicalPatient class. 

##### get_json_form function: 

**Description**: Function takes in an entry (in this case, a Pandas dataframe row) and structures the information provided into a formalized JSON form as structured in the MedicalPatients class. 

**Input**: ***patient_symptom_entry***: Dataframe Pandas row (for now) of all information needed to be processed as JSON form 

**Output**: ***json_form***: String of formalized JSON form

In [7]:
import typing_extensions as typing

class MedicalPatient(typing.TypedDict): 
    zipcode: int
    age: int
    gender: str
    symptoms: list[str]

def get_json_form(patient_symptom_entry): 
    entry_str = patient_symptom_entry.to_string()
    json_response = client.models.generate_content(
        model='gemini-2.0-flash', 
        config=types.GenerateContentConfig(
            temperature=0.1, 
            response_mime_type="application/json", 
            response_schema=MedicalPatient
        ), 
        contents=entry_str
    )
    json_form = json_response.text
    return json_form

#### Structured Output/Controlled Generation

**Description**: I created the get_general_model_config function to return a Gemini-generated output based on the temperature, top_p, and maximum number of tokens to return inputs.

**Input**: 
* ***temperature***: double that reflects the degree of randomness in selecting tokens for output
* ***top_p***: double that is the maximum culmulative probability that the model can reach to select tokens as candidates
* ***max_output_tokens***: int that indicates the maximum number of tokens that should be provided and included in the input

**Output**: ***str_output***: String of Gemini-generated output based on input parameters provided

In [8]:
def get_general_model_config(temperature, top_p, max_output_tokens): 
    str_output = types.GenerateContentConfig(
        temperature=temperature, 
        top_p=top_p, 
        max_output_tokens=max_output_tokens
    )
    return str_output

### Generative AI Capability #2: Few-Shot Prompting

**Description**: I created the few_shot_func function to implement the few-shot prompting AI capability as code

**Input**: 
* ***model_input***: String that is the input (e.g. a JSON form) that the model takes in consideration of to provide an output response
* ***prompt***: String that is the input prompt to provide to the Gemini model to provide a response for; in few-shot prompting, the input prompt provides a few examples with their expected responses to the Gemini model so that the Gemini models knows what to produce for their output format-wise and what their output should look like
* ***config***: Python method that is configuration to be used for the Gemini model to produce output (*refer to get_general_model_config as example of Python method to use as possible parameter value*)

**Output**: ***str_output***: String of Gemini-generated output based on input parameters provided

In [9]:
def few_shot_func(model_input, prompt, config): 
    few_shot_response = client.models.generate_content(
        model='gemini-2.0-flash', 
        config=config,
        contents=[prompt, model_input]
    )
    str_response = few_shot_response.text
    return str_response

### Generative AI Capability #3: Grounding

**Description**: I created the grounding_func function to implement the grounding AI capability as code

**Input**: ***input_prompt***: String that is the input prompt to provide to the Gemini model to provide a response for

**Output**: ***str_response***: String that is the output provided by the Gemini model as response to the input prompt

In [10]:
def grounding_func(input_prompt): 
    config_with_search = types.GenerateContentConfig(
        tools=[types.Tool(google_search=types.GoogleSearch())],
    )
    response = client.models.generate_content(
        model='gemini-2.0-flash', 
        contents=input_prompt, 
        config=config_with_search,
    )
    rc = response.candidates[0]
    str_response = rc.content.parts[0].text
    return str_response

## EmergencyZIP AI Test Run

Kaggle Notebooks seem to not work or do well with Python input, so instead, we generated a dataset of patients with relevant demographic information and their symptoms through ChatGPT (so the dataset below is ChatGPT-generated), and then the code picks a random patient to test run the AI to make sure it actually works as intended. 

In [11]:
patient_symptom_df = pd.read_csv('/kaggle/input/patient-symptom-dataset/patient_symptom_data.csv')
patient_symptom_df

,PatientID,ZipCode,Age,Gender,Symptoms
0,P001,70112,32,Female,Joint pain;Runny nose;Loss of taste;Sore throat
1,P002,15213,87,Female,Cough;Fever;Fatigue;Chest pain;Joint pain
2,P003,46201,21,Non-binary,Swelling;Blurred vision;Rash
3,P004,20001,46,Male,Chest tightness;Sneezing;Fever;Breathing diffi...
4,P005,96813,38,Non-binary,Vomiting;Headache;Runny nose;Chest pain;Breath...
...,...,...,...,...,...
95,P096,92101,71,Non-binary,Low-grade fever;Joint pain;Headache
96,P097,20001,80,Female,Vomiting;Abdominal cramps;Chest tightness;Abdo...
97,P098,85001,77,Female,Loss of smell;Abdominal pain
98,P099,84101,90,Non-binary,Fatigue;Blurred vision


We utilize the randint function from the Random library to select a random patient to process information and provide responses for. We then call the get_json_form method to have the AI mimic filling out a patient form in a simple and abstract way. 

In [12]:
patient_index = random.randint(0, 99)
patient_json_form = get_json_form(patient_symptom_df.iloc[patient_index])
print(patient_json_form)

{
"zipcode": 55414,
"age": 37,
"gender": "Non-binary",
"symptoms": ["Palpitations", "Wheezing"]
}


In [13]:
model_config = get_general_model_config(0.1, 1, 500)

few_shot_prompt = """Parse JSON into a string request and only return output: 

EXAMPLE: 
{
"zipcode": 78664, 
"age": 30, 
"gender": "Female", 
"symptoms": ["Cough", "Fever", "Sore throat"]
}
"30-year old female patient from zipcode 78664 currently experiencing symptoms of cough, fever, and sore throat."

EXAMPLE: 
{
"zipcode": 77055, 
"age": 25, 
"gender": "Non-binary", 
"symptoms": ["Joint pain", "Loss of smell"]
}
"25-year old non-binary patient from zipcode 77055 currently experiencing symptoms of joint pain and loss of smell."

EXAMPLE: 
{
"zipcode": 78681, 
"age": 15, 
"gender" "Male", 
"symptoms": ["Fatigue", "Loss of Vision", "Headache", "Joint pain"]
}
"15-year old male patient from zipcode 78681 currently experiencing symptoms of fatigue, loss of vision, headache, and joint pain."
"""

patient_medical_request = few_shot_func(patient_json_form, few_shot_prompt, model_config)
print(patient_medical_request)

"37-year old non-binary patient from zipcode 55414 currently experiencing symptoms of palpitations and wheezing."



In [14]:
diagnosis_prompt = patient_medical_request + "What disease are they most likely to experience now? Only return the singular most likely disease, please."
diagnosis = grounding_func(diagnosis_prompt)
Markdown(diagnosis)

Based on the information provided and the search results, the most likely condition this patient is experiencing is **asthma**.

Here's why:

*   **Common Symptoms:** Palpitations and wheezing are both symptoms associated with asthma.
*   **Age:** While asthma can occur at any age, new-onset asthma or changes in asthma symptoms can occur in adults.
*   **Other Considerations:** While other conditions can cause these symptoms (anxiety, heart conditions, thyroid issues), asthma is a common respiratory condition that directly explains both palpitations and wheezing.
*   **Zip Code:** I checked the zip code 55414 and found that it is located in Minneapolis, MN. No specific conditions are related to this zip code.


In [15]:
model_config_two = get_general_model_config(0, 1, 50)

few_shot_prompt_two = """Parse JSON to return only the Zipcode: 

EXAMPLE: 
{
"zipcode": 78664, 
"age": 30, 
"gender": "Female", 
"symptoms": ["Cough", "Fever", "Sore throat"]
}
78664

EXAMPLE: 
{
"zipcode": 77055, 
"age": 25, 
"gender": "Non-binary", 
"symptoms": ["Joint pain", "Loss of smell"]
}
77055

EXAMPLE: 
{
"zipcode": 78681, 
"age": 15, 
"gender" "Male", 
"symptoms": ["Fatigue", "Loss of Vision", "Headache", "Joint pain"]
}
78681
"""

patient_zipcode = few_shot_func(patient_json_form, few_shot_prompt_two, model_config_two)
print(patient_zipcode)

55414



In [16]:
recommendations_prompt = diagnosis + 'Can you get me the nearest best-rated medical facilities with their addresses that can treat the disease for zip code ' + str(patient_zipcode) + '?'
recommendations = grounding_func(recommendations_prompt)
Markdown(recommendations)

Here are some of the best-rated medical facilities near the 55414 zip code that can treat asthma.

*   **Allergy & Asthma Specialists, P.A.**
    *   **Address:** 825 Nicollet Mall, Suite 1149, Minneapolis, MN 55402
    *   **Phone:** 612-338-3333
    *   **Note:** They have another location in St. Louis Park.
*   **M Health Fairview Clinics and Surgery Center - Minneapolis**
    *   **Address:** 909 Fulton Street South East, Minneapolis, MN
    *   **Note:** They have extended hours and multiple specialties.
*   **Allergy and Asthma Center of Minnesota**
    *   **Address:** Multiple locations in Maplewood, West St. Paul and Woodbury, MN

It's important to note that "best-rated" can be subjective and depend on the source of the ratings. I recommend checking patient reviews and ratings on multiple platforms to get a comprehensive view. You should also consider factors like insurance coverage, specific needs, and ease of access when making your decision.


In [17]:
payments_prompt = "For each of the locations recommended to me: " + recommendations + " what types of payments and insurance plans are accepted for patients coming in for treatment?"
payments = grounding_func(payments_prompt)
Markdown(payments)

It's essential to confirm directly with each clinic for the most up-to-date and accurate information regarding their accepted insurance plans and payment methods. Insurance coverage can vary, and it's always best to verify your specific plan's details. Here's a general overview based on the search results:

**1. Allergy & Asthma Specialists, P.A.:**

*   **Insurance:** They state that they "accept nearly all insurance carriers." It is recommended to call your insurance to verify if Allergy & Asthma Specialists, part of Family Allergy & Asthma, providers are in-network with your specific plan.
*   **Payment:** You should be prepared to pay your co-payment in full at the time of service.

**2. M Health Fairview Clinics and Surgery Center - Minneapolis:**

*   **Insurance:** M Health Fairview is in-network with many insurance plans, including America's PPO/HealthEZ, Blue Cross Blue Shield of Minnesota, CMS/Medicare, and more. However, it is strongly recommended to contact your insurer to verify that the specific location or provider you plan to visit is included in your network.
*   **Important Billing Information:** This location may contain multiple hospital-based clinics, and your insurance may cover a visit differently than a standalone clinic.

**3. Allergy and Asthma Center of Minnesota:**

*   **Insurance**: They accept Medicare MN MA (MHCP), Metropolitan Health Plan (MHP), Preferred One, HealthPartners, Cigna, Medica Selectcare, United Healthcare, Aetna BCBS of Minnesota, Humana, Tricare Ucare Fairview Physicians Association (FPA).
*   **Payment:** Bring your insurance cards. Physician referral forms, if required by insurance.



## Limitations and Possible Next Steps

While the AI works as intended, this is a very basic setup/implementation of the AI (to be honest, we were short on time to implement the more advanced features/capabilities to make the AI run more efficiently and impressively). Thus, possible next steps/future directions are as follows: 

## Potential Impact 

We intend for the AI to empower individuals to make informed decisions that is best for their health and to know more and better of the variety of medical options provided for them to take care of their health, especially in illness and bad health (thus, we hope that with this AI, individuals take the initiative to become healthier and more health-conscious individuals). We also hope that the AI can help improve health outcomes for every zipcode/postal code as possible. 